In [ ]:
import nomenclature
import pyam
import ixmp4

In [ ]:
definition = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
platform = ixmp4.Platform("unep-fi-internal")

In [ ]:
#intensity_args = (
#    (
#        [
#            "Emissions|CO2|Energy|Demand|Industry|Iron and Steel",
#            "Emissions|CO2|Industrial Processes|Iron and Steel"
#        ],
#        "Production|Iron and Steel|Steel",
#        "Emission Intensity|CO2|Industry|Iron and Steel",
#    ),    
#)

In [ ]:
intensity_args =  {
    "Emission Intensity|Kyoto Gases|Energy|Supply|Electricity":
        (
            "Emissions|Kyoto Gases|Energy|Supply|Electricity",
            "Secondary Energy|Electricity",
        ),
    "Emission Intensity|CO2|Energy|Supply|Electricity":
        (
            "Emissions|CO2|Energy|Supply|Electricity",
            "Secondary Energy|Electricity",
        )
}

In [ ]:
def append_to_list(lst, item):
    if pyam.utils.is_list_like(item):
        lst.extend(item)
    else:
        lst.append(item)

In [ ]:
intensity_variables = list()

for key, value in intensity_args.items():
    append_to_list(intensity_variables, value[0])
    append_to_list(intensity_variables, value[1])

    print(key)
    print(definition.variable[key].unit)
    for v in value:
        print(definition.variable[v].unit)

intensity_variables

In [ ]:
df = pyam.read_ixmp4(platform, variable=intensity_variables)

In [ ]:
df

In [ ]:
result_list = list()

for key, value in intensity_args.items():
    result_list.append(df.divide(*value, key, ignore_units=definition.variable[key].unit))

In [ ]:
result = pyam.concat(result_list)

In [ ]:
result.filter(region="World", **{"Climate Category|SCI 2025": "GW*"}).plot(color="Climate Category|SCI 2025")